In [2]:
import numpy as np
from sklearn.cluster import KMeans

In [3]:
# tensor = np.load('../2015-5appliances-true-agg.npy')
ag_tensor = np.load('../2015-5appliances.numpy.npy')
true_tensor = np.load("../2015-5appliances-true-agg.npy")
from sklearn.metrics import mean_absolute_error

In [4]:
ag_agg = ag_tensor[:, 0, :, :]
true_agg = true_tensor[:, 0, :, :]
residual_agg = true_agg - ag_agg

In [5]:
residual_agg = residual_agg.reshape(68, -1)

In [6]:
kmeans = KMeans(n_clusters=3, random_state=0).fit(residual_agg)
kmeans.labels_

array([1, 1, 2, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 2,
       0, 1, 1, 0, 1, 0, 2, 1, 1, 1, 1, 0, 1, 1, 0, 1, 2, 1, 1, 1, 1, 0, 2,
       2, 0, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 0, 1], dtype=int32)

In [7]:
cluster = {}
for i in range(3):
    cluster[i] = [x for x in range(68) if kmeans.labels_[x] == i]

In [31]:
sc_with_pred = np.load("../code/baseline/result/DSC-pred.npy").item()[2]

In [32]:
pred = sc_with_pred[0]
for i in range(1, 5):
    pred = np.vstack([pred, sc_with_pred[i]])

In [28]:
APPLIANCE_ORDER = ['aggregate', 'hvac', 'fridge', 'dr', 'dw', 'mw']


In [40]:
error = {APPLIANCE_ORDER[i+1]:mean_absolute_error(pred[:, i,:,:].flatten(), 
                                                               true_tensor[:, i, :, :].flatten()) for i in range(pred.shape[1])}
print(error)

{'hvac': 1199.1295280102504, 'fridge': 795.25735564418096, 'dr': 79.902179731301345, 'dw': 80.053259664482624, 'mw': 31.11132486276021}


In [37]:
for i in range(3):
    print(len(cluster[i]))
    cluster_pred = pred[cluster[i]]
    cluster_gt = true_tensor[cluster[i]]
    cluster_pred = np.minimum(cluster_pred, cluster_gt[:, 0:1, :, :])
    error = {APPLIANCE_ORDER[i+1]:mean_absolute_error(cluster_pred[:, i,:,:].flatten(), 
                                                               cluster_gt[:, i, :, :].flatten()) for i in range(cluster_pred.shape[1])}
    print(error)

12
{'hvac': 1490.8661539744789, 'fridge': 952.322154481859, 'dr': 87.603601510192505, 'dw': 89.294082041208341, 'mw': 36.838615441234779}
48
{'hvac': 849.65506297095487, 'fridge': 714.23743443746446, 'dr': 78.214919687500142, 'dw': 71.07112171213987, 'mw': 29.562563445106136}
8
{'hvac': 2858.3713792996823, 'fridge': 1045.7796846279614, 'dr': 78.473607325771766, 'dw': 120.08485381345062, 'mw': 31.812957500972864}
